# Setup

After creating a conda env with r packages

```bash
conda create --name r_env r r-base r-essentials r-dplyr r-ggplot2 r-r6
conda activate r_env
conda install -c conda-forge r-irkernel jupyterlab r-devtools r-rpostgres
```

I opened a new terminal, opened up jupyter lab on a different port than my main jupyter lab server, and opened an R notebook.

In [3]:
library(R6)

In [2]:
library(DBI)

In [10]:
ls("package:R6")

[1] "is.R6"      "is.R6Class" "R6Class"

In [11]:
ls(getNamespace("R6"), all.names = TRUE)

[1] ".__NAMESPACE__."         ".__S3MethodsTable__."   
 [3] ".packageName"            "as.list.R6"             
 [5] "capsule"                 "encapsulate"            
 [7] "format.R6"               "format.R6ClassGenerator"
 [9] "generator_funs"          "indent"                 
[11] "is.R6"                   "is.R6Class"             
[13] "object_summaries"        "plot.R6"                
[15] "print.R6"                "print.R6ClassGenerator" 
[17] "R6Class"                 "trim"

In [14]:
PGConnection <- R6Class("PGConnection",
    public = list(
        conn = NULL,
        initialize = function(dbname, host, port, user, password) {
            self$conn <- dbConnect(RPostgres::Postgres(), dbname=dbname, host=host, port=port, user=user, password=password)
            message("Connection established.")
        },
        finalize = function() {
            if(!is.null(self$conn) && dbIsValid(self$conn)) {
                dbDisconnect(self$conn)
                message("Connection closed.")
            }
        }
    )
)

In [18]:
pg_conn = PGConnection$new(
    dbname="db_name",
    host="localhost",
    user="db_user",
    pass="db_password",
    port="5432"
)

Connection established.



In [12]:
# ?R6